In [131]:
import numpy as np
from scipy.constants import Boltzmann as k
from numpy.typing import NDArray
from typing import List

In [132]:
L = 9
def generate_grid(L: int) -> np.ndarray:
    return  np.random.choice([-1,1], size=(L,L))
state = generate_grid(9)

In [133]:
print(state)

[[-1  1  1  1  1 -1 -1 -1  1]
 [-1  1  1  1 -1 -1  1  1  1]
 [-1 -1 -1  1 -1 -1  1  1  1]
 [ 1  1 -1 -1  1 -1  1 -1  1]
 [ 1  1  1 -1 -1 -1  1  1  1]
 [ 1  1 -1 -1  1 -1 -1 -1  1]
 [ 1 -1  1 -1 -1 -1  1 -1 -1]
 [ 1 -1 -1  1 -1 -1 -1 -1 -1]
 [-1 -1  1  1 -1  1 -1  1  1]]


In [134]:
def Energy(spins: np.ndarray) -> int:
    H = 0
    for row in spins:
        H += np.sum(row[:-1] * row[1:])
        #print(row[:-1] * row[1:])
        #print(H)
    
    for col in spins.T:
        H += np.sum(col[:-1] * col[1:])
        #print(col[:-1] * col[1:])
        #print(H)

    return H


In [135]:
print(Energy(state))

16


In [136]:
def DeltaE(initial_spins: np.ndarray, spin_to_flip: tuple[int, int]) -> int:
    #Calculates E_new - E_old
    L = initial_spins.shape[0]
    sum_of_neighbors = 0
    x,y = spin_to_flip[0], spin_to_flip[1]
    for dx in [-1,1]:
        if (0 <= x + dx < L):
             sum_of_neighbors += initial_spins[x + dx, y]

    for dy in [-1,1]:
        if (0 <= y + dy < L):
            sum_of_neighbors += initial_spins[x, y + dy]
     
    return -2 * sum_of_neighbors * initial_spins[x, y]


In [137]:
print(DeltaE(state, [4,4]))

0


In [138]:
new_state = state.copy()
new_state[4,4] = state[4,4] * -1

In [139]:
Energy(state)

16

In [140]:
Energy(new_state)

16

In [141]:
all_ones = np.ones((4,4))
print(all_ones)

[[1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]]


In [142]:
print(Energy(all_ones))

24.0


In [143]:
flip_all_ones = all_ones.copy()
flip_all_ones[1,0] = -1

In [144]:
print(Energy(flip_all_ones))

18.0


In [195]:
print(DeltaE(all_ones, [1,0]))

-6.0


In [197]:
def move_spins(config: np.ndarray, spin: tuple[int,int]) -> np.ndarray:
    #given a configuration c, return a new configuration c'
    spin_to_flip_x, spin_to_flip_y = spin
    new_config = config.copy()
    new_config[spin_to_flip_x, spin_to_flip_y] = config[spin_to_flip_x, spin_to_flip_y] * -1
    return new_config

In [198]:
def mcmc(proposal_func, init_dist, score_func,
                        num_iters, steps=30):
    """
    Runs the metropolis-hastings algorithm for
    num_iters iterations, using proposal_func
    to generate candidate states and scorer to
    assign probability scores to candidate
    states.
    
    proposal_func: function that proposes
        candidate state; takes in current state as
        argument and returns candidate state
    init_func: function that proposes starting
        state; takes no arguments and returns a
        sample state
    score_func: function that calculates f(y)/f(x)
        * g(y,x)/g(x,y); takes in two state samples
        (the current sample x then the candidate y).
    
    Returns a sequence of every step-th sample. You 
    should only sample on upon acceptance of a new
    proposal. Do not keep sampling the current state.
    
    Note the total number of samples will NOT be
    equal to num_iters. num_iters is the total number
    of proposals we generate.
    """
    samples = []
    sample = init_dist
    for sweep in range(num_iters):
        for step in range(steps):
            candidate = proposal_func(sample)
            acceptance_prob = min(1, score_func(sample, candidate))
            if np.random.uniform() < acceptance_prob:
                sample = move_spins(sample, candidate)
                if sweep > 20:
                    samples.append(sample.flatten())
    return samples[::step]

In [199]:
def flip_spin(config: np.ndarray) -> tuple[int, int]:
    L = config.shape[0]
    return np.random.randint(0, L, size = (1,2))[0]

In [204]:
temp = 300
beta = 1

In [205]:
def score_fn(config: np.ndarray, proposed_flip: tuple[int, int]) -> float:
    return np.exp(-beta * DeltaE(config, proposed_flip))

In [206]:
print(all_ones)
print(move_spins(all_ones))

[[1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]]


TypeError: move_spins() missing 1 required positional argument: 'spin'

In [207]:
mcmc(flip_spin, generate_grid(3), score_fn, 5_000)

[array([ 1, -1, -1, -1,  1, -1,  1, -1,  1]),
 array([ 1, -1,  1, -1,  1, -1,  1, -1,  1]),
 array([ 1, -1,  1, -1,  1, -1, -1, -1,  1]),
 array([ 1, -1, -1, -1, -1,  1,  1, -1, -1]),
 array([-1,  1,  1,  1, -1, -1, -1,  1, -1]),
 array([-1,  1,  1,  1, -1,  1, -1,  1, -1]),
 array([-1,  1, -1,  1, -1,  1, -1,  1, -1]),
 array([-1,  1, -1,  1, -1,  1, -1,  1,  1]),
 array([ 1, -1, -1,  1, -1,  1, -1,  1, -1]),
 array([-1,  1,  1,  1, -1,  1, -1,  1, -1]),
 array([-1,  1,  1,  1, -1, -1, -1,  1, -1]),
 array([ 1, -1,  1, -1,  1, -1,  1, -1,  1]),
 array([ 1,  1,  1, -1,  1, -1,  1, -1,  1]),
 array([ 1, -1, -1, -1,  1,  1,  1, -1,  1]),
 array([-1, -1,  1, -1,  1, -1, -1, -1, -1]),
 array([ 1, -1,  1, -1,  1, -1, -1, -1, -1]),
 array([-1, -1,  1, -1,  1, -1,  1, -1,  1]),
 array([ 1, -1,  1, -1,  1, -1, -1,  1,  1]),
 array([ 1, -1,  1, -1,  1,  1,  1, -1,  1]),
 array([ 1, -1,  1, -1,  1, -1,  1, -1,  1]),
 array([ 1, -1,  1, -1,  1, -1, -1, -1,  1]),
 array([ 1,  1, -1, -1,  1, -1,  1